In [1]:
%cd ..\..\..
import pandas as pd
import gc
import utilities

D:\Projects\CondaAnalyses\PMI


In [2]:
data_path = 'data_checkpoints/usecase_cab/'

In [3]:
# import the base df
df = pd.read_csv(data_path+'base_df.csv', index_col=0)
df['time']= pd.to_datetime(df['time'])
df.head()

,id,file_name,latitude,longitude,time,occupancy
0,abboip_1211033530,abboip,37.75153,-122.39447,2008-05-17 14:12:10,0
1,abboip_1211033614,abboip,37.75149,-122.39447,2008-05-17 14:13:34,0
2,abboip_1211033674,abboip,37.75149,-122.39447,2008-05-17 14:14:34,0
3,abboip_1211033735,abboip,37.75149,-122.39446,2008-05-17 14:15:35,0
4,abboip_1211035303,abboip,37.75144,-122.39449,2008-05-17 14:41:43,0


In [4]:
# drop columns not needed for the travelled distance
df.drop(['file_name', 'occupancy'], axis=1, inplace=True)
df.head(2)

,id,latitude,longitude,time
0,abboip_1211033530,37.75153,-122.39447,2008-05-17 14:12:10
1,abboip_1211033614,37.75149,-122.39447,2008-05-17 14:13:34


In [5]:
# read events_df for future join
events_df = pd.read_csv(data_path+'occupancy_state_change.csv', index_col=0)
events_df.head(2)

,id,id_event
0,abboip_1211033530,abboip_1211033530
1,abboip_1211033614,abboip_1211033530


In [6]:
# Join both dataframes to get travelled distance
df = df.merge(events_df, on='id')
del events_df
gc.collect()
df.head()

,id,latitude,longitude,time,id_event
0,abboip_1211033530,37.75153,-122.39447,2008-05-17 14:12:10,abboip_1211033530
1,abboip_1211033614,37.75149,-122.39447,2008-05-17 14:13:34,abboip_1211033530
2,abboip_1211033674,37.75149,-122.39447,2008-05-17 14:14:34,abboip_1211033530
3,abboip_1211033735,37.75149,-122.39446,2008-05-17 14:15:35,abboip_1211033530
4,abboip_1211035303,37.75144,-122.39449,2008-05-17 14:41:43,abboip_1211033530


In [7]:
# append origin and destination coordinates
df = utilities.append_origin_destination_coordinates(
    df,
    'time',
    'id_event',
    'latitude',
    'longitude',
)
df.head()

,id,latitude,longitude,time,id_event,origin_latitude,destination_latitude,origin_longitude,destination_longitude
0,abboip_1211033530,37.75153,-122.39447,2008-05-17 14:12:10,abboip_1211033530,37.75153,37.74978,-122.39447,-122.39709
1,abboip_1211033614,37.75149,-122.39447,2008-05-17 14:13:34,abboip_1211033530,37.75153,37.74978,-122.39447,-122.39709
2,abboip_1211033674,37.75149,-122.39447,2008-05-17 14:14:34,abboip_1211033530,37.75153,37.74978,-122.39447,-122.39709
3,abboip_1211033735,37.75149,-122.39446,2008-05-17 14:15:35,abboip_1211033530,37.75153,37.74978,-122.39447,-122.39709
4,abboip_1211035303,37.75144,-122.39449,2008-05-17 14:41:43,abboip_1211033530,37.75153,37.74978,-122.39447,-122.39709


In [8]:
df.drop(['latitude', 'longitude', 'time', 'id_event'], axis=1, inplace=True)
df.head()

,id,origin_latitude,destination_latitude,origin_longitude,destination_longitude
0,abboip_1211033530,37.75153,37.74978,-122.39447,-122.39709
1,abboip_1211033614,37.75153,37.74978,-122.39447,-122.39709
2,abboip_1211033674,37.75153,37.74978,-122.39447,-122.39709
3,abboip_1211033735,37.75153,37.74978,-122.39447,-122.39709
4,abboip_1211035303,37.75153,37.74978,-122.39447,-122.39709


In [9]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
origin_latitude,11220058.0,37.761178,0.062434,34.71052,37.76162,37.78379,37.79180,50.30546
destination_latitude,11220058.0,37.761580,0.060122,34.71052,37.76109,37.78356,37.79156,50.30546
origin_longitude,11220058.0,-122.412645,0.041115,-125.33431,-122.42907,-122.41212,-122.40081,-122.00002
destination_longitude,11220058.0,-122.413575,0.035794,-126.16715,-122.42809,-122.41140,-122.40040,-122.00070


In [10]:
df.to_csv(data_path+'origin_destination_coordinates.csv')
del df
gc.collect()

0